<a href="https://colab.research.google.com/github/MahdiZaman/Spring2019-CAP5610/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**************HW 2**

---



The goal of this homework is to create a convolutional neural network for the CIFAR10 data set. See this colab notebook how to load the CIFAR data in Keras.

You should not use any pretrained convnets that come with Keras. You have to create and train your own convnets with Keras from scratch.



*   Simple hold-out validation

Make sure that the data is divided into:

training set (80%)
validation set (20%)
test set.
Use the training set to train your neural networks. Evaluate their performance on the validation data set.

After trying several different architectures, choose the one that performs best of the validation set. Try at least four different architectures by using data augmentation, using dropout, varying the number of layers, the number of filters, etc.

Train this final architecture on the data from the training set and validation set and evaluate its performance on the test set.

*   k-fold validation

Reevaluate your best architecture using k-fold validation with k=5, that is, the size of the validation fold is 20%. Does the accuracy/loss obtain by k-fold validation differ from the accuracy/loss obtain by simple hold-out validation.

**Importing Necessary Libraries and Loading Dataset**

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator


from keras.datasets import cifar10
(train_images_original, train_labels_original), (test_images_original, test_labels) = cifar10.load_data()
# (train_images_original, train_labels_original), (test_images_original, test_labels_n) = cifar10.load_data() #categorical

Using TensorFlow backend.


170500096/170498071 [==============================] - 20s 0us/step


**Reshaping and Normalizing Training and Test Samples**

In [0]:
train_images_rshp = train_images_original.reshape((train_images_original.shape[0], train_images_original.shape[1], train_images_original.shape[2], train_images_original.shape[3]))
test_images_rshp = test_images_original.reshape((test_images_original.shape[0], test_images_original.shape[1], test_images_original.shape[2], test_images_original.shape[3]))

train_images_n = train_images_rshp.astype('float32') / 255.0
test_images = test_images_rshp.astype('float32') / 255.0

***Simple Hold-out Validation***

In [0]:
indices = np.random.permutation(train_images_original.shape[0])
val_indcs = indices[0:10000]
val_images = train_images_n[val_indcs]
val_labels = train_labels_original[val_indcs]
# val_labels_n = train_labels_original[val_indcs] #categorical


train_indcs = indices[10000:]
train_images = train_images_n[train_indcs]
train_labels = train_labels_original[train_indcs]
# train_labels_n = train_labels_original[train_indcs] #categorical

In [0]:
# train_labels = np_utils.to_categorical(train_labels_n, 10)
# val_labels = np_utils.to_categorical(val_labels_n, 10)
# test_labels = np_utils.to_categorical(test_labels_n, 10)

In [0]:
#Initializations

weight_decay = 1e-4  #For Kernel Regularizers
epochs = 80
batch_size=32



1.) **Baseline Model**





In [0]:
# set up the layers
## Baseline

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    #
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    #
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    #
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()


# compile the model

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# train the model
history = model.fit(train_images, 
                      train_labels, 
                      epochs=50,  
                      validation_data=(val_images, val_labels))

2.) **Deeper Architecture w/ More Layers and Filters, Drop-Out, and L2 Regularizer**

---



In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape= (32,32,3), strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.25),

    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.30),
    
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.35),
    
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.40),
  
    keras.layers.Conv2D(512, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(512, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.50),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

# compile the model

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# train the model
epochs = 50
history = model.fit(train_images, 
                      train_labels, 
                      epochs=epochs,  
                      validation_data=(val_images, val_labels))

**3) Adding Data Augmentation**

---



In [0]:
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(train_images)

In [6]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape= (32,32,3), strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.25),

    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.30),
    
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.35),
    
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.40),
  
    keras.layers.Conv2D(512, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.Conv2D(512, (3,3), activation='relu', padding='same', strides= (1,1), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay)),
    keras.layers.MaxPooling2D((2, 2), padding= 'same'),
    keras.layers.Dropout(0.50),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)    

In [0]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# model.fit_generator(datagen.flow(train_images, train_labels, batch_size=32),\
#                     steps_per_epoch=train_images.shape[0] // batch_size, epochs=50,\
#                     verbose=1,validation_data=(val_images,val_labels))

model.fit_generator(datagen.flow(train_images, train_labels, batch_size=32), steps_per_epoch=train_images.shape[0] // batch_size, epochs= 50, verbose=1, validation_data=(val_images,val_labels))

Epoch 1/50
 186/1250 [===>..........................] - ETA: 1:10 - loss: 2.6733 - acc: 0.1198